## GR5245 Homework 2

(a) Download the TensorFlow flowers dataset from tensorflow_datasets. Take the first 80% as 
the training set and the remaining 20% as the validation set. 

In [1]:
import tensorflow as tf
import tensorflow_datasets as tfds
import numpy as np
import matplotlib.pyplot as plt


In [2]:
(x_train, x_valid), info = tfds.load('tf_flowers',
                                     split=['train[:80%]',
                                            'train[80%:]'],
                                     with_info=True,
                                     as_supervised=True)


(b) Write a preprocessing function to resize the images (both in training and validation sets) to have 
64x64 pixel values and rescale all pixel values to have values between 0 and 1. Build a data pipeline 
which will generate mini-batches of 32 samples randomly selected from a buffer of 1000 items. 

In [3]:
def preprocess(image, label):
    image = tf.image.resize(image, size=(64, 64))/255.0
    return image, label


In [4]:
train_ds = x_train.map(preprocess).shuffle(1000).batch(32)
valid_ds = x_valid.map(preprocess).shuffle(1000).batch(32)

(c) Create a Keras model as shown in the table below. Compile the model using SGD with a constant 
learning rate of 0.001, sparse categorical entropy as loss function and sparse categorical accuracy as 
the metric. Train the model for 20 epochs using the data pipeline prepared in (b). Keep the training 
history for later use.  

In [5]:
model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(32, 3, 1, padding="same", activation="relu"),
    tf.keras.layers.MaxPool2D(2, 1, padding="valid"),
    tf.keras.layers.Conv2D(64, 3, 1, padding="same", activation="relu"),
    tf.keras.layers.MaxPool2D(2, 1, padding="valid"),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(64, activation="relu"),
    tf.keras.layers.Dense(5, activation="softmax")
])


In [6]:
model.compile(optimizer=tf.keras.optimizers.SGD(learning_rate=0.001),
              loss=tf.keras.losses.SparseCategoricalCrossentropy(),
              metrics=[tf.keras.metrics.SparseCategoricalAccuracy()])

In [7]:
history = model.fit(train_ds,
                    validation_data=valid_ds,
                    epochs=20)

Epoch 1/20


2022-10-25 12:20:27.946197: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


92/92 [==============================] - 11s 113ms/step - loss: 1.5599 - sparse_categorical_accuracy: 0.3178 - val_loss: 1.5123 - val_sparse_categorical_accuracy: 0.3174
Epoch 2/20
92/92 [==============================] - 10s 105ms/step - loss: 1.4303 - sparse_categorical_accuracy: 0.4152 - val_loss: 1.3665 - val_sparse_categorical_accuracy: 0.4700
Epoch 3/20
92/92 [==============================] - 10s 108ms/step - loss: 1.3245 - sparse_categorical_accuracy: 0.4629 - val_loss: 1.3008 - val_sparse_categorical_accuracy: 0.4496
Epoch 4/20
92/92 [==============================] - 10s 104ms/step - loss: 1.2488 - sparse_categorical_accuracy: 0.4894 - val_loss: 1.2812 - val_sparse_categorical_accuracy: 0.4482
Epoch 5/20
92/92 [==============================] - 10s 105ms/step - loss: 1.1979 - sparse_categorical_accuracy: 0.4973 - val_loss: 1.1888 - val_sparse_categorical_accuracy: 0.5123
Epoch 6/20
92/92 [==============================] - 10s 104ms/step - loss: 1.1648 - sparse_categorical_acc

(d) Create a new model by adding batch normalization layers as shown below. Compile and train the 
model using the same hyper parameters as in (c). Keep the training history for later use. 

In [ ]:
model2 = tf.keras.Sequential([
    tf.keras.layers.Conv2D(32, 3, 1, padding="same", use_bias=False),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.ReLU(),
    tf.keras.layers.MaxPool2D(2, 1, padding="valid"),

    tf.keras.layers.Conv2D(64, 3, 1, padding="same", use_bias=False),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.ReLU(),
    tf.keras.layers.MaxPool2D(2, 1, padding="valid"),

    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(64, use_bias=False),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.ReLU(),
    tf.keras.layers.Dense(5, activation="softmax")
])
